# Process IMERG data

combine IMERG files into a single dataset, swapping dimensions so it is (time,lat,lon), restrict region, remove extra variables, write out reduced file

In [5]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, colors
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from scipy.interpolate import interp2d, RectBivariateSpline
from datetime import datetime, timedelta
import pandas as pd
import cftime
import netCDF4 as nc
import glob
import os

In [2]:
def get_IMERG_path(datetime_in):
    IMERG_path_FMT = '/huracan/tank1/work/ORCESTRA/imerg/late_run_V07/3B-HHR-L.MS.MRG.3IMERG.{year:04d}{month:02d}{day:02d}-S{hour0:02d}{minute0:02d}{second0:02d}-E{hour1:02d}{minute1:02d}{second1:02d}.{minday:04d}.V07B.RT-H5'

    datetime_0 = datetime_in
    datetime_1 = datetime_in + timedelta(seconds=1799)

    year_0, month_0, day_0, hour_0, minute_0, second_0 = datetime_0.year, datetime_0.month, datetime_0.day, datetime_0.hour, datetime_0.minute, datetime_0.second
    year_1, month_1, day_1, hour_1, minute_1, second_1 = datetime_1.year, datetime_1.month, datetime_1.day, datetime_1.hour, datetime_1.minute, datetime_1.second

    minday = int(np.rint((datetime_0 - datetime(year_0,month_0,day_0,0,0,0)).seconds / 60.))

    IMERG_path = IMERG_path_FMT.format(year=year_0,month=month_0,day=day_0,hour0=hour_0,minute0=minute_0,second0=second_0,hour1=hour_1,minute1=minute_1,second1=second_1,minday=minday)
    
    return IMERG_path

In [3]:
# Read IMERG data using xarray
IMERG = xr.open_dataset(get_IMERG_path(datetime(2024,8,16,0,0,0)),engine='h5netcdf',group='Grid')

# Swap lat & lon dimensions, so it is (time,lat,lon)
IMERG = IMERG.transpose('time','lat','lon',...)

# Restrict the region
IMERG = IMERG.sel(lat=slice(-10,50),lon=slice(-90,60))

In [7]:
IMERG

<xarray.Dataset> Size: 14MB
Dimensions:                         (time: 1, lon: 1500, lat: 600, nv: 2,
                                     lonv: 2, latv: 2)
Coordinates:
  * time                            (time) object 8B 2024-08-16 00:00:00
  * lon                             (lon) float32 6kB -89.95 -89.85 ... 59.95
  * lat                             (lat) float32 2kB -9.95 -9.85 ... 49.95
Dimensions without coordinates: nv, lonv, latv
Data variables:
    time_bnds                       (time, nv) object 16B ...
    lon_bnds                        (lon, lonv) float32 12kB ...
    lat_bnds                        (lat, latv) float32 5kB ...
    precipitation                   (time, lat, lon) float32 4MB ...
    randomError                     (time, lat, lon) float32 4MB ...
    probabilityLiquidPrecipitation  (time, lat, lon) float32 4MB ...
    precipitationQualityIndex       (time, lat, lon) float32 4MB ...
Attributes:
    GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegistration=CENTER;\nLatitudeRe...

In [ ]:
plt.contourf(IMERG.lon,IMERG.lat,IMERG.precipitation[0,:,:])
cbar = plt.colorbar()

In [ ]:
# Load IMERG data
file_paths = glob.glob("/huracan/tank1/work/ORCESTRA/imerg/late_run_V07/3B-HHR-L.MS.MRG.3IMERG*")

#Open the files and combine them into a single dataset
IMERG_ds = xr.open_mfdataset(file_paths,engine='h5netcdf',group='Grid',combine='by_coords')

# Swap lat & lon dimensions, so it is (time,lat,lon)
IMERG_ds = IMERG_ds.transpose('time','lat','lon',...)

# Restrict the region
IMERG_ds = IMERG_ds.sel(lat=slice(-10,50),lon=slice(-90,60))

# Remove extra variables
IMERG_reduced = IMERG_ds.drop_vars(['probabilityLiquidPrecipitation','randomError','precipitationQualityIndex'])

#Write out to new netcdf file
IMERG_reduced.to_netcdf('/huracan/tank1/work/ORCESTRA/imerg/imerg_20240809.nc')


KeyboardInterrupt: 

In [12]:
IMERG_reduced

<xarray.Dataset> Size: 11GB
Dimensions:        (time: 2928, nv: 2, lon: 1500, lonv: 2, lat: 600, latv: 2)
Coordinates:
  * time           (time) object 23kB 2024-08-01 00:00:00 ... 2024-09-30 23:3...
  * lon            (lon) float32 6kB -89.95 -89.85 -89.75 ... 59.75 59.85 59.95
  * lat            (lat) float32 2kB -9.95 -9.85 -9.75 ... 49.75 49.85 49.95
Dimensions without coordinates: nv, lonv, latv
Data variables:
    time_bnds      (time, nv) object 47kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lon_bnds       (time, lon, lonv) float32 35MB dask.array<chunksize=(1, 1500, 2), meta=np.ndarray>
    lat_bnds       (time, lat, latv) float32 14MB dask.array<chunksize=(1, 600, 2), meta=np.ndarray>
    precipitation  (time, lat, lon) float32 11GB dask.array<chunksize=(1, 600, 115), meta=np.ndarray>
Attributes:
    GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegistration=CENTER;\nLatitudeRe...